# PASTIS matrix from E-fields

This notebook calculates PASTIS matrices for the low, mid, and high order modes from single-mode E-fields in the focal plane.  
It also calculates matrices on the low order wavefront sensor (LOWFS) and out of band wavefront sensor (OBWFS).

In [1]:
import os
import time
from shutil import copy

from astropy.io import fits
import astropy.units as u
import hcipy
import numpy as np

import pastis.util as util    
from pastis.config import CONFIG_PASTIS 
from pastis.e2e_simulators.luvoir_imaging import LuvoirA_APLC 

## Initial setup and parameters

Set up data paths for input and output

In [2]:
#root_dir = CONFIG_PASTIS.get('local', 'local_data_path')
root_dir = "/Users/asahoo/Desktop/data_repos/multi_mode"
repo_dir = "/Users/asahoo/repos/PASTIS"
coronagraph_design = 'small' # user provides
overall_dir = util.create_data_path(root_dir, telescope='luvoir_'+coronagraph_design)
resDir = os.path.join(overall_dir, 'matrix_numerical')

print(resDir)

/Users/asahoo/Desktop/data_repos/multi_mode

/Users/asahoo/Desktop/data_repos/multi_mode/2022-01-10T15-46-14_luvoir_small/matrix_numerical


Read from configfile how many modes each DM should be able to do

In [3]:
max_LO = CONFIG_PASTIS.getint('dm_objects', 'number_of_low_order_modes')
max_MID = CONFIG_PASTIS.getint('dm_objects', 'number_of_mid_order_modes')
max_HI = CONFIG_PASTIS.getint('dm_objects', 'number_of_high_order_modes')
num_DM_act = CONFIG_PASTIS.getint('dm_objects', 'number_of_continuous_dm_actuators')

print(f'max_LO: {max_LO}')
print(f'max_MID: {max_MID}')
print(f'max_HI: {max_HI}')
print(f'num_DM_act: {num_DM_act}')

max_LO: 15
max_MID: 1
max_HI: 4
num_DM_act: 4


Read some more required parameters from the configfile

In [4]:
nb_seg = CONFIG_PASTIS.getint('LUVOIR', 'nb_subapertures')
wvln = CONFIG_PASTIS.getfloat('LUVOIR', 'lambda') * 1e-9  # m
diam = CONFIG_PASTIS.getfloat('LUVOIR', 'diameter')  # m
nm_aber = CONFIG_PASTIS.getfloat('LUVOIR', 'calibration_aberration') * 1e-9   # m
sampling = CONFIG_PASTIS.getfloat('LUVOIR', 'sampling')

In [5]:
print('LUVOIR apodizer design: {}'.format(coronagraph_design)) 
print()
print('Wavelength: {} m'.format(wvln))
print('Telescope diameter: {} m'.format(diam))
print('Number of segments: {}'.format(nb_seg))
print()
print('Sampling: {} px per lambda/D'.format(sampling))

LUVOIR apodizer design: small

Wavelength: 5.000000000000001e-07 m
Telescope diameter: 15.0 m
Number of segments: 120

Sampling: 4.0 px per lambda/D


In [6]:
# Create necessary directories if they don't exist yet
os.makedirs(resDir, exist_ok=True)
os.makedirs(os.path.join(resDir, 'OTE_images'), exist_ok=True)
os.makedirs(os.path.join(resDir, 'psfs'), exist_ok=True)

In [7]:
#  Copy configfile to resulting matrix directory
util.copy_config(resDir)

Saving the configfile to outputs folder.


In [8]:
# Create LUVOIR simulator instance
optics_input = os.path.join(util.find_repo_location(), CONFIG_PASTIS.get('LUVOIR', 'optics_path_in_repo'))
luvoir = LuvoirA_APLC(optics_input, coronagraph_design, sampling) 

/opt/miniconda3/envs/pastis/lib/python3.7/site-packages/hcipy/field/cartesian_grid.py:74: UserWarning: No automatic weights could be calculated for this grid.
  self.weights *= np.abs(scale)**self.ndim


Generate the deformable mirrors that are required.

**This will take quite some time**

In [9]:
luvoir.create_global_zernike_mirror(max_LO)
luvoir.create_segmented_mirror(max_MID)
luvoir.create_ripple_mirror(max_HI)
luvoir.create_continuous_deformable_mirror(num_DM_act)

In [10]:
# Figur out the total number of "actuators" (= effective modes) for each DM
n_LO = luvoir.zernike_mirror.num_actuators 
n_MID = luvoir.sm.num_actuators 
n_HI = luvoir.ripple_mirror.num_actuators 
n_DM = luvoir.dm.num_actuators

print(f'n_LO: {n_LO}')
print(f'n_MID: {n_MID}')
print(f'n_HI: {n_HI}')
print(f'n_DM: {n_DM}')

n_LO: 15
n_MID: 120
n_HI: 16
n_DM: 16


Define some parameters that are needed for the subsampled LOWFS output.

In [11]:
z_pup_downsample = CONFIG_PASTIS.getfloat('numerical', 'z_pup_downsample') 
N_pup_z = int(luvoir.pupil_grid.shape[0] / z_pup_downsample)
grid_zernike = hcipy.field.make_pupil_grid(N_pup_z, diameter=luvoir.diam)

## Flatten all DMs and create unaberrated reference PSF

In [12]:
LO_modes = np.zeros(n_LO)
MID_modes = np.zeros(n_MID)
HI_modes = np.zeros(n_HI)
DM_modes = np.zeros(n_DM)

In [13]:
luvoir.zernike_mirror.actuators = LO_modes 
luvoir.sm.actuators = MID_modes
luvoir.ripple_mirror.actuators = HI_modes 
luvoir.dm.actuators = DM_modes

In [14]:
# Calculate the unaberrated coro and direct PSFs in INTENSITY
unaberrated_coro_psf, ref = luvoir.calc_psf(ref=True, display_intermediate=False)

In [15]:
# Define the normalization factor for contrast units
norm = np.max(ref)

In [16]:
# Calculate the unaberrated coro and direct PSFs in E-FIELDS
nonaberrated_coro_psf, ref, efield = luvoir.calc_psf(ref=True, display_intermediate=False, return_intermediate='efield')
Efield_ref = nonaberrated_coro_psf.electric_field

## Generate LOW-order PASTIS matrix from E-fields

In [17]:
print('Generating the E-fields for LOW modes in science plane')
print(f'number of LO modes: {n_LO}')

Generating the E-fields for LOW modes in science plane
number of LO modes: 15


In [18]:
# Create lists that will hold measured fields
print(f'Calibration aberration used: {nm_aber} m')
start_time = time.time()
focus_fieldS = []
focus_fieldS_Re = []
focus_fieldS_Im = []

Calibration aberration used: 1e-09 m


In [19]:
for pp in range(0, n_LO):
    print(f'Working on mode {pp}/{n_LO}')
    
    # Apply calibration aberration to used mode
    LO_modes = np.zeros(n_LO)
    LO_modes[pp] = nm_aber / 2
    luvoir.zernike_mirror.actuators  = LO_modes
    
    # Calculate coronagraphic E-field and add to lists
    aberrated_coro_psf, inter = luvoir.calc_psf(display_intermediate=False, return_intermediate='efield')
    focus_field1 = aberrated_coro_psf
    focus_fieldS.append(focus_field1)
    focus_fieldS_Re.append(focus_field1.real)
    focus_fieldS_Im.append(focus_field1.imag)

Working on mode 0/15
Working on mode 1/15
Working on mode 2/15
Working on mode 3/15
Working on mode 4/15
Working on mode 5/15
Working on mode 6/15
Working on mode 7/15
Working on mode 8/15
Working on mode 9/15
Working on mode 10/15
Working on mode 11/15
Working on mode 12/15
Working on mode 13/15
Working on mode 14/15


In [20]:
# Construct the PASTIS matrix from the E-fields
mat_fast = np.zeros([n_LO, n_LO])   # create empty matrix

for i in range(0, n_LO):
    for j in range(0, n_LO):
        test = np.real((focus_fieldS[i].electric_field - Efield_ref) * np.conj(focus_fieldS[j].electric_field - Efield_ref))
        dh_test = (test / norm) * luvoir.dh_mask
        contrast = np.mean(dh_test[np.where(luvoir.dh_mask != 0)])
        mat_fast[i, j] = contrast

In [21]:
# Normalize by the calibration aberration
matrix_pastis = np.copy(mat_fast)
matrix_pastis /= np.square(nm_aber * 1e9)

Save results out to disk

In [22]:
filename_matrix = 'PASTISmatrix_num_LO_' + str(max_LO)
hcipy.write_fits(matrix_pastis, os.path.join(resDir, filename_matrix + '.fits'))
print('Matrix saved to:', os.path.join(resDir, filename_matrix + '.fits'))

Matrix saved to: /Users/asahoo/Desktop/data_repos/multi_mode/2022-01-10T15-46-14_luvoir_small/matrix_numerical/PASTISmatrix_num_LO_15.fits


In [23]:
filename_matrix = 'EFIELD_Re_matrix_num_LO_' + str(max_LO)
hcipy.write_fits(focus_fieldS_Re, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Real saved to:', os.path.join(resDir, filename_matrix + '.fits'))

Efield Real saved to: /Users/asahoo/Desktop/data_repos/multi_mode/2022-01-10T15-46-14_luvoir_small/matrix_numerical/EFIELD_Re_matrix_num_LO_15.fits


In [24]:
filename_matrix = 'EFIELD_Im_matrix_num_LO_' + str(max_LO)
hcipy.write_fits(focus_fieldS_Im, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Imag saved to:', os.path.join(resDir, filename_matrix + '.fits'))

Efield Imag saved to: /Users/asahoo/Desktop/data_repos/multi_mode/2022-01-10T15-46-14_luvoir_small/matrix_numerical/EFIELD_Im_matrix_num_LO_15.fits


In [25]:
end_time = time.time()

In [26]:
print('Runtime for LO modes:', end_time - start_time, 'sec =', (end_time - start_time) / 60, 'min')
print('Data saved to {}'.format(resDir))

Runtime for LO modes: 4.401622772216797 sec = 0.07336037953694662 min
Data saved to /Users/asahoo/Desktop/data_repos/multi_mode/2022-01-10T15-46-14_luvoir_small/matrix_numerical


## Generate MID-order PASTIS matrix from E-fields

In [27]:
print('Generating the Efield for MID modes in science plane')
print(f'number of MID modes: {n_MID}')

Generating the Efield for MID modes in science plane
number of MID modes: 120


In [28]:
LO_modes = np.zeros(n_LO)
MID_modes = np.zeros(n_MID)
HI_modes = np.zeros(n_HI)

In [29]:
# Create lists that will hold measured fields
print(f'Calibration aberration used: {nm_aber} m')
start_time = time.time()
focus_fieldS = []
focus_fieldS_Re = []
focus_fieldS_Im = []

Calibration aberration used: 1e-09 m


In [30]:
for pp in range(0, n_MID):
    print(f'Working on mode {pp}/{n_MID}')
    
    # Apply calibration aberration to used mode
    MID_modes = np.zeros(n_MID)
    MID_modes[pp] = nm_aber / 2
    luvoir.sm.actuators  = MID_modes
    
    # Calculate coronagraphic E-field and add to lists
    aberrated_coro_psf, inter = luvoir.calc_psf(display_intermediate=False, return_intermediate='efield')
    focus_field1 = aberrated_coro_psf
    focus_fieldS.append(focus_field1)
    focus_fieldS_Re.append(focus_field1.real)
    focus_fieldS_Im.append(focus_field1.imag)

Working on mode 0/120
Working on mode 1/120
Working on mode 2/120
Working on mode 3/120
Working on mode 4/120
Working on mode 5/120
Working on mode 6/120
Working on mode 7/120
Working on mode 8/120
Working on mode 9/120
Working on mode 10/120
Working on mode 11/120
Working on mode 12/120
Working on mode 13/120
Working on mode 14/120
Working on mode 15/120
Working on mode 16/120
Working on mode 17/120
Working on mode 18/120
Working on mode 19/120
Working on mode 20/120
Working on mode 21/120
Working on mode 22/120
Working on mode 23/120
Working on mode 24/120
Working on mode 25/120
Working on mode 26/120
Working on mode 27/120
Working on mode 28/120
Working on mode 29/120
Working on mode 30/120
Working on mode 31/120
Working on mode 32/120
Working on mode 33/120
Working on mode 34/120
Working on mode 35/120
Working on mode 36/120
Working on mode 37/120
Working on mode 38/120
Working on mode 39/120
Working on mode 40/120
Working on mode 41/120
Working on mode 42/120
Working on mode 43/12

In [31]:
# Construct the PASTIS matrix from the E-fields
mat_fast = np.zeros([n_MID, n_MID])

for i in range(0, n_MID):
    for j in range(0, n_MID):
        test = np.real((focus_fieldS[i].electric_field - Efield_ref) * np.conj(focus_fieldS[j].electric_field - Efield_ref))
        dh_test = (test / norm) * luvoir.dh_mask
        contrast = np.mean(dh_test[np.where(luvoir.dh_mask != 0)])
        mat_fast[i, j] = contrast

In [32]:
# Normalize by the calibration aberration
matrix_pastis = np.copy(mat_fast)
matrix_pastis /= np.square(nm_aber * 1e9)  

In [33]:
filename_matrix = 'PASTISmatrix_num_MID_' + str(max_MID)
hcipy.write_fits(matrix_pastis, os.path.join(resDir, filename_matrix + '.fits'))
print('Matrix saved to:', os.path.join(resDir, filename_matrix + '.fits'))

Matrix saved to: /Users/asahoo/Desktop/data_repos/multi_mode/2022-01-10T15-46-14_luvoir_small/matrix_numerical/PASTISmatrix_num_MID_1.fits


In [34]:
filename_matrix = 'EFIELD_Re_matrix_num_MID_' + str(max_MID)
hcipy.write_fits(focus_fieldS_Re, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Real saved to:', os.path.join(resDir, filename_matrix + '.fits'))

Efield Real saved to: /Users/asahoo/Desktop/data_repos/multi_mode/2022-01-10T15-46-14_luvoir_small/matrix_numerical/EFIELD_Re_matrix_num_MID_1.fits


In [35]:
filename_matrix = 'EFIELD_Im_matrix_num_MID_' + str(max_MID)
hcipy.write_fits(focus_fieldS_Im, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Imag saved to:', os.path.join(resDir, filename_matrix + '.fits'))

Efield Imag saved to: /Users/asahoo/Desktop/data_repos/multi_mode/2022-01-10T15-46-14_luvoir_small/matrix_numerical/EFIELD_Im_matrix_num_MID_1.fits


In [36]:
end_time = time.time()
print('Runtime for MID modes:', end_time - start_time, 'sec =', (end_time - start_time) / 60, 'min')
print('Data saved to {}'.format(resDir))

Runtime for MID modes: 40.70173215866089 sec = 0.6783622026443481 min
Data saved to /Users/asahoo/Desktop/data_repos/multi_mode/2022-01-10T15-46-14_luvoir_small/matrix_numerical


## Generate MID-order PASTIS matrix from E-fields

In [37]:
print('Generating the Efield for HI modes in science plane')
print(f'number of HI modes: {n_HI}')

Generating the Efield for HI modes in science plane
number of HI modes: 16


In [38]:
LO_modes = np.zeros(n_LO)
MID_modes = np.zeros(n_MID)
HI_modes = np.zeros(n_HI)

In [39]:
# Create lists that will hold measured fields
print(f'Calibration aberration used: {nm_aber} m')
start_time = time.time()
focus_fieldS = []
focus_fieldS_Re = []
focus_fieldS_Im = []

Calibration aberration used: 1e-09 m


In [40]:
for pp in range(0, n_HI):
    print(f'Working on mode {pp}/{n_HI}')
    
    # Apply calibration aberration to used mode
    HI_modes = np.zeros(n_HI)
    HI_modes[pp] = nm_aber / 2
    luvoir.ripple_mirror.actuators = HI_modes
    
    # Calculate coronagraphic E-field and add to lists
    aberrated_coro_psf, inter = luvoir.calc_psf(display_intermediate=False, return_intermediate='efield')
    focus_field1 = aberrated_coro_psf
    focus_fieldS.append(focus_field1)
    focus_fieldS_Re.append(focus_field1.real)
    focus_fieldS_Im.append(focus_field1.imag)

Working on mode 0/16
Working on mode 1/16
Working on mode 2/16
Working on mode 3/16
Working on mode 4/16
Working on mode 5/16
Working on mode 6/16
Working on mode 7/16
Working on mode 8/16
Working on mode 9/16
Working on mode 10/16
Working on mode 11/16
Working on mode 12/16
Working on mode 13/16
Working on mode 14/16
Working on mode 15/16


In [41]:
# Construct the PASTIS matrix from the E-fieldsmat_fast = np.zeros([n_HI, n_HI])
for i in range(0, n_HI):
    for j in range(0, n_HI):
        test = np.real((focus_fieldS[i].electric_field - Efield_ref) * np.conj(focus_fieldS[j].electric_field - Efield_ref))
        dh_test = (test / norm) * luvoir.dh_mask
        contrast = np.mean(dh_test[np.where(luvoir.dh_mask != 0)])
        mat_fast[i, j] = contrast

In [42]:
# Normalize by the calibration aberration
matrix_pastis = np.copy(mat_fast) 
matrix_pastis /= np.square(nm_aber * 1e9)

In [43]:
filename_matrix = 'PASTISmatrix_num_HI_' + str(max_HI)
hcipy.write_fits(matrix_pastis, os.path.join(resDir, filename_matrix + '.fits'))
print('Matrix saved to:', os.path.join(resDir, filename_matrix + '.fits'))

Matrix saved to: /Users/asahoo/Desktop/data_repos/multi_mode/2022-01-10T15-46-14_luvoir_small/matrix_numerical/PASTISmatrix_num_HI_4.fits


In [44]:
filename_matrix = 'EFIELD_Re_matrix_num_HI_' + str(max_HI)
hcipy.write_fits(focus_fieldS_Re, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Real saved to:', os.path.join(resDir, filename_matrix + '.fits'))

Efield Real saved to: /Users/asahoo/Desktop/data_repos/multi_mode/2022-01-10T15-46-14_luvoir_small/matrix_numerical/EFIELD_Re_matrix_num_HI_4.fits


In [45]:
filename_matrix = 'EFIELD_Im_matrix_num_HI_' + str(max_HI)
hcipy.write_fits(focus_fieldS_Im, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Imag saved to:', os.path.join(resDir, filename_matrix + '.fits'))

Efield Imag saved to: /Users/asahoo/Desktop/data_repos/multi_mode/2022-01-10T15-46-14_luvoir_small/matrix_numerical/EFIELD_Im_matrix_num_HI_4.fits


In [46]:
end_time = time.time()
print('Runtime for HI modes:', end_time - start_time, 'sec =', (end_time - start_time) / 60, 'min')
print('Data saved to {}'.format(resDir))

Runtime for HI modes: 4.831820964813232 sec = 0.08053034941355387 min
Data saved to /Users/asahoo/Desktop/data_repos/multi_mode/2022-01-10T15-46-14_luvoir_small/matrix_numerical


## Generate LOW-order matrix on LOWFS

In [47]:
print('Generating the Efield for LOW modes through LOWFS')
print('number of LO modes'.format(n_LO))

Generating the Efield for LOW modes through LOWFS
number of LO modes


Flatten DMs

In [48]:
LO_modes = np.zeros(n_LO)
MID_modes = np.zeros(n_MID)
HI_modes = np.zeros(n_HI)

In [49]:
luvoir.zernike_mirror.actuators = LO_modes 
luvoir.sm.actuators = MID_modes
luvoir.ripple_mirror.actuators = HI_modes 

In [50]:
zernike_ref = luvoir.calc_low_order_wfs()

Calculate unaberrated reference E-field on Zernike WFS on a subsampled grid.

In [51]:
zernike_ref_sub_real = hcipy.field.subsample_field(zernike_ref.real, z_pup_downsample, grid_zernike, statistic='mean')
zernike_ref_sub_imag = hcipy.field.subsample_field(zernike_ref.imag, z_pup_downsample, grid_zernike, statistic='mean')
Efield_ref = zernike_ref_sub_real + 1j*zernike_ref_sub_imag

In [52]:
# Create lists that will hold measured fields
print(f'Calibration aberration used: {nm_aber} m')
start_time = time.time()
focus_fieldS = []
focus_fieldS_Re = []
focus_fieldS_Im = []

Calibration aberration used: 1e-09 m


In [53]:
for pp in range(0, n_LO):
    print(f'Working on mode {pp}/{n_LO}')
    
    # Apply calibration aberration to used mode
    LO_modes = np.zeros(n_LO)
    LO_modes[pp] = nm_aber / 2
    luvoir.zernike_mirror.actuators = LO_modes
    
    # Calculate E-field on Zernike WFS and add to lists
    zernike_meas = luvoir.calc_low_order_wfs()
    zernike_meas_sub_real = hcipy.field.subsample_field(zernike_meas.real, z_pup_downsample, grid_zernike,statistic='mean')
    zernike_meas_sub_imag = hcipy.field.subsample_field(zernike_meas.imag, z_pup_downsample, grid_zernike,statistic='mean')
    focus_field1 = zernike_meas_sub_real + 1j * zernike_meas_sub_imag
    focus_fieldS.append(focus_field1)
    focus_fieldS_Re.append(focus_field1.real)
    focus_fieldS_Im.append(focus_field1.imag)

Working on mode 0/15
Working on mode 1/15
Working on mode 2/15
Working on mode 3/15
Working on mode 4/15
Working on mode 5/15
Working on mode 6/15
Working on mode 7/15
Working on mode 8/15
Working on mode 9/15
Working on mode 10/15
Working on mode 11/15
Working on mode 12/15
Working on mode 13/15
Working on mode 14/15


In [54]:
filename_matrix = 'EFIELD_LOWFS_Re_matrix_num_LO_' + str(max_LO)
hcipy.write_fits(focus_fieldS_Re, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Real saved to:', os.path.join(resDir, filename_matrix + '.fits'))

Efield Real saved to: /Users/asahoo/Desktop/data_repos/multi_mode/2022-01-10T15-46-14_luvoir_small/matrix_numerical/EFIELD_LOWFS_Re_matrix_num_LO_15.fits


In [55]:
filename_matrix = 'EFIELD_LOWFS_Im_matrix_num_LO_' + str(max_LO)
hcipy.write_fits(focus_fieldS_Im, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Imag saved to:', os.path.join(resDir, filename_matrix + '.fits'))

Efield Imag saved to: /Users/asahoo/Desktop/data_repos/multi_mode/2022-01-10T15-46-14_luvoir_small/matrix_numerical/EFIELD_LOWFS_Im_matrix_num_LO_15.fits


In [56]:
end_time = time.time()
print('Runtime for LO modes and LOWFS:', end_time - start_time, 'sec =', (end_time - start_time) / 60, 'min')
print('Data saved to {}'.format(resDir))

Runtime for LO modes and LOWFS: 4.106564998626709 sec = 0.06844274997711182 min
Data saved to /Users/asahoo/Desktop/data_repos/multi_mode/2022-01-10T15-46-14_luvoir_small/matrix_numerical


## Generate MID-order matrix on OBWFS

In [57]:
print('Generating the Efield for MID modes to OBWFS')
print('number of MID modes'.format(n_MID))

Generating the Efield for MID modes to OBWFS
number of MID modes


In [58]:
# Flatten DMs
LO_modes = np.zeros(n_LO)
MID_modes = np.zeros(n_MID)
HI_modes = np.zeros(n_HI)

luvoir.zernike_mirror.actuators = LO_modes
luvoir.sm.actuators = MID_modes
luvoir.ripple_mirror.actuators = HI_modes

In [59]:
# Calculate unaberrated reference E-field on Zernike WFS on a subsampled grid.
zernike_ref = luvoir.calc_out_of_band_wfs()
zernike_ref_sub_real = hcipy.field.subsample_field(zernike_ref.real, z_pup_downsample, grid_zernike, statistic='mean')
zernike_ref_sub_imag = hcipy.field.subsample_field(zernike_ref.imag, z_pup_downsample, grid_zernike, statistic='mean')
Efield_ref = zernike_ref_sub_real + 1j*zernike_ref_sub_imag

In [60]:
# Create lists that will hold measured fields
print(f'Calibration aberration used: {nm_aber} m')
start_time = time.time()
focus_fieldS = []
focus_fieldS_Re = []
focus_fieldS_Im = []

Calibration aberration used: 1e-09 m


In [61]:
for pp in range(0, n_MID):
    print(f'Working on mode {pp}/{n_MID}')
    
    # Apply calibration aberration to used mode
    MID_modes = np.zeros(n_MID)
    MID_modes[pp] = nm_aber / 2
    luvoir.sm.actuators = MID_modes
    
    # Calculate E-field on OBWFS and add to lists
    zernike_meas = luvoir.calc_out_of_band_wfs()
    zernike_meas_sub_real = hcipy.field.subsample_field(zernike_meas.real, z_pup_downsample, grid_zernike, statistic='mean')
    zernike_meas_sub_imag = hcipy.field.subsample_field(zernike_meas.imag, z_pup_downsample, grid_zernike, statistic='mean')
    focus_field1 = zernike_meas_sub_real + 1j * zernike_meas_sub_imag
    focus_fieldS.append(focus_field1)
    focus_fieldS_Re.append(focus_field1.real)
    focus_fieldS_Im.append(focus_field1.imag)

Working on mode 0/120
Working on mode 1/120
Working on mode 2/120
Working on mode 3/120
Working on mode 4/120
Working on mode 5/120
Working on mode 6/120
Working on mode 7/120
Working on mode 8/120
Working on mode 9/120
Working on mode 10/120
Working on mode 11/120
Working on mode 12/120
Working on mode 13/120
Working on mode 14/120
Working on mode 15/120
Working on mode 16/120
Working on mode 17/120
Working on mode 18/120
Working on mode 19/120
Working on mode 20/120
Working on mode 21/120
Working on mode 22/120
Working on mode 23/120
Working on mode 24/120
Working on mode 25/120
Working on mode 26/120
Working on mode 27/120
Working on mode 28/120
Working on mode 29/120
Working on mode 30/120
Working on mode 31/120
Working on mode 32/120
Working on mode 33/120
Working on mode 34/120
Working on mode 35/120
Working on mode 36/120
Working on mode 37/120
Working on mode 38/120
Working on mode 39/120
Working on mode 40/120
Working on mode 41/120
Working on mode 42/120
Working on mode 43/12

In [62]:
filename_matrix = 'EFIELD_OBWFS_Re_matrix_num_MID_' + str(max_MID)
hcipy.write_fits(focus_fieldS_Re, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Real saved to:', os.path.join(resDir, filename_matrix + '.fits'))

Efield Real saved to: /Users/asahoo/Desktop/data_repos/multi_mode/2022-01-10T15-46-14_luvoir_small/matrix_numerical/EFIELD_OBWFS_Re_matrix_num_MID_1.fits


In [63]:
filename_matrix = 'EFIELD_OBWFS_Im_matrix_num_MID_' + str(max_MID)
hcipy.write_fits(focus_fieldS_Im, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Imag saved to:', os.path.join(resDir, filename_matrix + '.fits'))

Efield Imag saved to: /Users/asahoo/Desktop/data_repos/multi_mode/2022-01-10T15-46-14_luvoir_small/matrix_numerical/EFIELD_OBWFS_Im_matrix_num_MID_1.fits


In [64]:
end_time = time.time()
print('Runtime for MID modes and OBWFS:', end_time - start_time, 'sec =', (end_time - start_time) / 60, 'min')
print('Data saved to {}'.format(resDir))

Runtime for MID modes and OBWFS: 31.805800914764404 sec = 0.5300966819127401 min
Data saved to /Users/asahoo/Desktop/data_repos/multi_mode/2022-01-10T15-46-14_luvoir_small/matrix_numerical
